In [16]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

False

In [17]:
os.environ["OPENAI_API_KEY"] = "api_key"

In [18]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [19]:
loader = DirectoryLoader('f2', glob="**/*.txt", show_progress=True)

docs = loader.load()

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 32.90it/s]


In [20]:
print (f"You have {len(docs)} essays loaded")

You have 1 essays loaded


In [21]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

print (f"Your {len(docs)} documents have been split into {len(splits)} chunks")

Your 1 documents have been split into 1 chunks


In [22]:
if 'vectordb' in globals(): 
    vectordb.delete_collection()

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
import logging

In [24]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [25]:
question = "Please predict the possible outcome and possible ways to resolve the case"
llm = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [26]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the potential outcomes of the case and what are some possible solutions to resolve it?', '2. Can you provide me with different scenarios for the outcome of the case and suggest potential resolutions?', '3. I would like to know the various possible results of the case and explore different approaches to resolve it. Could you help me with that?']


In [27]:
len(unique_docs)

1

In [28]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [30]:
ans = llm.predict(text=PROMPT.format_prompt(context=unique_docs,question=question).text)

In [31]:
ans

'Based on the information provided, the possible outcome of the case could be a favorable judgment for Ms. Jane Doe if she is able to provide sufficient evidence supporting her claims of gender-based discrimination and retaliation. If the court finds that TechCo Corporation indeed discriminated against Ms. Doe and wrongfully terminated her employment, possible ways to resolve the case could include financial compensation for damages, reinstatement of her employment, or a settlement agreement between the parties. However, without further information or legal analysis, it is difficult to determine the exact outcome or resolution of the case.'